In [15]:
from bertviz import head_view
import torch
from medbert.features.mlm_plos_dataset import MLM_PLOS_Dataset
from medbert.features.embeddings import BertEmbeddings
from medbert.common import pytorch

In [50]:
inputs = torch.load("..\\data\\tokenized\\example_data.pt")
vocab = torch.load("..\\data\\vocab\\example_data.pt")
inv_vocab = {v:k for k,v in vocab.items()}
model = torch.load("..\\models\\mlm_plos_pretrained\\test\\model.pt")
model.eval()

max_len = 50
dataset = MLM_PLOS_Dataset(inputs, vocab, max_len=max_len)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)
input = [d for i,d in enumerate(dataloader) if i<10][0]
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
config = model.config
embeddings = BertEmbeddings(config=config)
# input = pytorch.batch_to_device(input, device)
embedding_output = embeddings(input['codes'], input['segments'])
# process
outputs = model(inputs_embeds=embedding_output, 
            attention_mask=input['attention_mask'], 
            labels=input['labels'],
            next_sentence_label=input['plos'],
            output_attentions=True) 
attention = outputs[-1] 
input_tokens = [inv_vocab[i] for i in inputs['codes'][0]]
while len(input_tokens) < max_len:
    input_tokens.append('PAD')
print(input_tokens)
head_view(attention, input_tokens)

['Y24.6', 'O02.8', 'Q40.7', 'K59.4', 'M88.6', 'U48.6', 'D38.5', 'U14.2', 'N40.4', 'K91.8', 'V57.7', 'A29.8', 'C22.9', 'MASK', 'L85.7', 'W60.8', 'E89.9', 'W15.5', 'K17.7', 'E51.7', 'C40.9', 'E68.5', 'C74.1', 'O28.7', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']


<IPython.core.display.Javascript object>